# Parallelizing image convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-09-20 06:42:07,229	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 148.4 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-09-20 06:42:45,338	INFO worker.py:1786 -- Started a local Ray instance.


Python version:,3.11.10
Ray version:,2.36.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 86.0, speedup = 1.73


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 173.2, speedup = 0.86


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:39719' processes=4 threads=4, memory=15.62 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-09-20 06:47:09,191 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4532a06ea15536dc9fbe0ca1c4d733be
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.52899275, -0.52427405,  0.89770557, -0.35720198],
       [-2.70633575, -1.10276735,  2.45281879,  1.3708096 ],
       [-0.13947925,  1.89313211, -1.19474475,  0.75201314],
       [-0.44877652,  0.91503085,  0.26433525,  0.84484219]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:09,286 - distributed.worker - ERROR - Compute Failed
Key:       fconv-490d8cf1b450658d4c2aaf35b3686138
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 06:47:09,474 - distributed.worker - ERROR - Compute Failed
Key:       fconv-b5be2ac6552f2c56056cfa2caf5680a8
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.11163485, -1.24699731, -1.62796614, -0.37306861],
       [-0.8772471 , -0.19632645, -2.20540546, -0.19588019],
       [ 1.45792265,  0.16478257, -1.14016664,  0.04631598],
       [ 0.59164355, -0.04683107,  0.5424388 ,  2.05470644]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:09,564 - distributed.worker - ERROR - Compute Failed
Key:       fconv-c16c00110a932291847d078d99861703
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 06:47:09,764 - distributed.worker - ERROR - Compute Failed
Key:       fconv-d65007ce2e0e018c866566057febc87a
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.98826299,  0.99806792,  2.28055338, -1.50815329],
       [ 0.65679599, -1.20878961, -1.1781562 , -0.84330078],
       [-0.0261701 , -1.08832652, -1.60227746, -1.06041489],
       [-0.45327593, -0.85540029,  0.32723852, -0.07108255]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:09,856 - distributed.worker - ERROR - Compute Failed
Key:       fconv-d41daacb69d5ef0711a04442e74d5073
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 06:47:10,051 - distributed.worker - ERROR - Compute Failed
Key:       fconv-e0090e5c270363bb1238caed0dfda969
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.31797753,  0.38908825,  1.58100326, -0.05047756],
       [-1.1967979 , -0.27896225, -1.47514997,  0.10825964],
       [ 0.21093185, -0.45748411,  0.67272069, -0.23243961],
       [ 0.05610073,  1.29165519,  0.55556176,  1.21244187]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:10,145 - distributed.worker - ERROR - Compute Failed
Key:       fconv-862f83634da63def63074035ee60dbdf
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 06:47:10,332 - distributed.worker - ERROR - Compute Failed
Key:       fconv-dd63fd8f0ef7d31f62f9d6af5020a74f
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.52899275, -0.52427405,  0.89770557, -0.35720198],
       [-2.70633575, -1.10276735,  2.45281879,  1.3708096 ],
       [-0.13947925,  1.89313211, -1.19474475,  0.75201314],
       [-0.44877652,  0.91503085,  0.26433525,  0.84484219]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:10,430 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ba51a005430b2168ac7dd8d86359369b
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-09-20 06:47:45,857 - distributed.worker - ERROR - Compute Failed
Key:       fconv-67d30f16c9e73f8135e101d129e770f3
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.98826299,  0.99806792,  2.28055338, -1.50815329],
       [ 0.65679599, -1.20878961, -1.1781562 , -0.84330078],
       [-0.0261701 , -1.08832652, -1.60227746, -1.06041489],
       [-0.45327593, -0.85540029,  0.32723852, -0.07108255]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:45,949 - distributed.worker - ERROR - Compute Failed
Key:       fconv-e5320dc6eacf9e1ede197d0fe1a73472
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 06:47:46,138 - distributed.worker - ERROR - Compute Failed
Key:       fconv-238f8463baa9fc645b1dcaa8791456e4
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.31797753,  0.38908825,  1.58100326, -0.05047756],
       [-1.1967979 , -0.27896225, -1.47514997,  0.10825964],
       [ 0.21093185, -0.45748411,  0.67272069, -0.23243961],
       [ 0.05610073,  1.29165519,  0.55556176,  1.21244187]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:46,232 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2ddfd2891e5077ae8efbcbe7d67c2d68
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 06:47:46,422 - distributed.worker - ERROR - Compute Failed
Key:       fconv-c0565ba5d76233e1a01cc32e8cadc111
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.52899275, -0.52427405,  0.89770557, -0.35720198],
       [-2.70633575, -1.10276735,  2.45281879,  1.3708096 ],
       [-0.13947925,  1.89313211, -1.19474475,  0.75201314],
       [-0.44877652,  0.91503085,  0.26433525,  0.84484219]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:46,521 - distributed.worker - ERROR - Compute Failed
Key:       fconv-02d64a34f7ad13f132c5bcb8f404859c
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 38.2, speedup = 3.88


2024-09-20 06:47:46,710 - distributed.worker - ERROR - Compute Failed
Key:       fconv-9eec08cfdf4d2edd4737b6adb0201573
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.11163485, -1.24699731, -1.62796614, -0.37306861],
       [-0.8772471 , -0.19632645, -2.20540546, -0.19588019],
       [ 1.45792265,  0.16478257, -1.14016664,  0.04631598],
       [ 0.59164355, -0.04683107,  0.5424388 ,  2.05470644]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 06:47:46,814 - distributed.worker - ERROR - Compute Failed
Key:       fconv-28703358f87600f40ee825a3c2976b7c
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

In [16]:
client.close()

2024-09-20 06:47:46,857 - distributed.worker - ERROR - Compute Failed
Key:       fconv-d1e5b048ce109477c61d94b60832bbd2
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.31797753,  0.38908825,  1.58100326, -0.05047756],
       [-1.1967979 , -0.27896225, -1.47514997,  0.10825964],
       [ 0.21093185, -0.45748411,  0.67272069, -0.23243961],
       [ 0.05610073,  1.29165519,  0.55556176,  1.21244187]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''



## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io